# ETL Data Processor - Retail Sales Data Mart

## Project Overview
This project demonstrates ETL processing for a retail sales dimensional data mart, extracting data from multiple sources (MySQL OLTP, CSV files, MongoDB), transforming it, and loading it into a star schema optimized for analytical queries.

**Business Process**: Retail sales transactions with customer, product, and temporal dimensions


## 1. Data Mart Design 

**Dimensional Model Components:**
- **DimCustomer**: Customer dimension with demographics
- **DimProduct**: Product dimension with categories and pricing  
- **DimDate**: Date dimension for temporal analysis
- **FactSales**: Sales transaction fact table
- **FactSales_Margin**: Enriched fact table with margin calculations

**Data Sources Used:**
- MySQL OLTP database (src_customers, src_products)
- CSV files (customers.csv, products.csv, dates.csv, sales.csv)
- MongoDB (product_costs.json)


### Setup and Configuration


In [31]:
import pandas as pd
import pymongo
import certifi
import json
import os
from sqlalchemy import create_engine, text


In [32]:
mysql_args = {
    "uid" : "root",
    "pwd" : "thevidu",
    "hostname" : "localhost",
    "dbname" : "northwind_dw2"
}

mongodb_args = {
    "user_name" : "m001-student",
    "password" : "password",
    "cluster_name" : "cluster_name",
    "cluster_subnet" : "xxxxx",
    "cluster_location" : "local", # "local"
    "db_name" : "northwind_purchasing"
}

## 2. ETL Pipeline Implementation (Requirement 2)

### 2a. Extract from SQL Database (Requirement 2a)
**Source**: MySQL OLTP tables (src_customers, src_products)


In [33]:
def get_sql_dataframe(sql_query, **args):
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(text(sql_query), connection)
    connection.close()
    return dframe

def set_dataframe(df, table_name, pk_column, db_operation, **args):
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    connection.close()

def get_mongo_client(**args):
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
    return client

def get_mongo_dataframe(mongo_client, db_name, collection, query):
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    return dframe

def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
    mongo_client.close()

def get_sql_conn(**args):
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    return connection

In [34]:
SQL_DIR  = os.path.join(os.getcwd(), "sql")
DATA_DIR = os.path.join(os.getcwd(), 'data')

### Transform and Load Dimensions
**Transformation**: Modify column structure and integrate data from multiple sources


In [35]:
conn = get_sql_conn(**mysql_args)
schema_path = os.path.join(SQL_DIR, "create_datamart.sql")
with open(schema_path, "r") as f:
    ddl = f.read()

statements = [stmt.strip() for stmt in ddl.split(';') if stmt.strip()]
for statement in statements:
    if statement:  # Skip empty statements
        conn.execute(text(statement))
        
print("Executed create_datamart.sql")

Executed create_datamart.sql


### Build and Load Fact Table
**Integration**: Combine dimension keys with transaction data


In [36]:
conn = get_sql_conn(**mysql_args)

conn.execute(text("""
CREATE TABLE IF NOT EXISTS src_customers (
  customer_id VARCHAR(20) PRIMARY KEY,
  first_name VARCHAR(50),
  last_name VARCHAR(50),
  email VARCHAR(100),
  city VARCHAR(50),
  state VARCHAR(50),
  country VARCHAR(50),
  phone VARCHAR(20)   -- extra column we will DROP in transform
);
"""))

conn.execute(text("""
CREATE TABLE IF NOT EXISTS src_products (
  product_id VARCHAR(20) PRIMARY KEY,
  product_name VARCHAR(100),
  category VARCHAR(50),
  sub_category VARCHAR(50),
  list_price DECIMAL(10,2),
  supplier_code VARCHAR(20)  -- extra column we will DROP in transform
);
"""))

conn.execute(text("DELETE FROM src_customers"))
conn.execute(text("DELETE FROM src_products"))

conn.execute(text("""
INSERT INTO src_customers (customer_id, first_name, last_name, email, city, state, country, phone) VALUES
('CUST-001','Ava','Nguyen','ava.nguyen@example.com','Austin','TX','USA','512-555-1001'),
('CUST-002','Liam','Patel','liam.patel@example.com','Seattle','WA','USA','206-555-1002'),
('CUST-003','Sophia','Kim','sophia.kim@example.com','New York','NY','USA','212-555-1003'),
('CUST-004','Noah','Garcia','noah.garcia@example.com','Miami','FL','USA','305-555-1004'),
('CUST-005','Mia','Johnson','mia.johnson@example.com','Chicago','IL','USA','773-555-1005');
"""))

conn.execute(text("""
INSERT INTO src_products (product_id, product_name, category, sub_category, list_price, supplier_code) VALUES
('SKU-100','Wireless Mouse','Electronics','Accessories',19.99,'SUP-01'),
('SKU-101','Mechanical Keyboard','Electronics','Accessories',79.50,'SUP-01'),
('SKU-102','USB-C Charger','Electronics','Power',24.00,'SUP-02'),
('SKU-200','Water Bottle 1L','Home & Kitchen','Drinkware',12.75,'SUP-03'),
('SKU-300','Notebook Set (3)','Office','Stationery',9.99,'SUP-03');
"""))

conn.close()
print("Created and seeded src_customers, src_products")


Created and seeded src_customers, src_products


### Extract from File System
**Source**: CSV files (customers.csv, products.csv, dates.csv, sales.csv)


In [ ]:
# load from csv files
customers_csv = os.path.join(DATA_DIR, "customers.csv")
products_csv  = os.path.join(DATA_DIR, "products.csv")
dates_csv     = os.path.join(DATA_DIR, "dates.csv")
sales_csv     = os.path.join(DATA_DIR, "sales.csv")


def stage_csv(csv_path, table_name):
    df = pd.read_csv(csv_path)
    set_dataframe(df, table_name, pk_column="__dummy__ignore__", db_operation="update", **mysql_args)  # append/create staging
    print(f"Staged {table_name} from {os.path.basename(csv_path)}")

In [38]:
conn = get_sql_conn(**mysql_args)

stage_csv(customers_csv, "stg_customers_file")
stage_csv(products_csv,  "stg_products_file")
stage_csv(dates_csv,     "stg_dates_file")
stage_csv(sales_csv,     "stg_sales_file")

Staged stg_customers_file from customers.csv
Staged stg_products_file from products.csv
Staged stg_dates_file from dates.csv
Staged stg_sales_file from sales.csv


In [ ]:
# Extract from SQL source system
src_customers_df = get_sql_dataframe("SELECT * FROM src_customers", **mysql_args)
src_products_df  = get_sql_dataframe("SELECT * FROM src_products",  **mysql_args)

### Customer Dimension ETL: combine, deduplicate, and load


In [40]:
# Combine source and file data for customers
file_customers_df = get_sql_dataframe("SELECT * FROM stg_customers_file", **mysql_args)

cust_all = (
    src_customers_df[["customer_id","first_name","last_name","email","city","state","country"]]
    .rename(columns={
        "customer_id":"CustomerID","first_name":"FirstName","last_name":"LastName",
        "email":"Email","city":"City","state":"State","country":"Country"
    })
    .drop_duplicates(subset=["CustomerID"])
)

# Bring in any customers present in the file but not in the source system
only_in_file = file_customers_df.merge(
    cust_all[["CustomerID"]], how="left", left_on="CustomerID", right_on="CustomerID", indicator=True
)
only_in_file = only_in_file[only_in_file["_merge"]=="left_only"][
    ["CustomerID","FirstName","LastName","Email","City","State","Country"]
]

dim_customer_df = pd.concat([cust_all, only_in_file], ignore_index=True)

dim_customer_df.to_sql("DimCustomer", con=conn, index=False, if_exists="append")
print(f"Loaded DimCustomer ({len(dim_customer_df)} rows)")

Loaded DimCustomer (5 rows)


### Product Dimension ETL


In [41]:
# Combine source and file data for products
file_products_df = get_sql_dataframe("SELECT * FROM stg_products_file", **mysql_args)

prod_all = (
    src_products_df[["product_id","product_name","category","sub_category","list_price"]]
    .rename(columns={
        "product_id":"ProductID","product_name":"ProductName",
        "category":"Category","sub_category":"SubCategory","list_price":"UnitPrice"
    })
    .drop_duplicates(subset=["ProductID"])
)

# Fill in any products only present in file
only_prod_in_file = file_products_df.merge(
    prod_all[["ProductID"]], how="left", on="ProductID", indicator=True
)
only_prod_in_file = only_prod_in_file[only_prod_in_file["_merge"]=="left_only"][
    ["ProductID","ProductName","Category","SubCategory","UnitPrice"]
]

### Load Dimension Tables
**Process**: Load DimProduct and DimDate tables with proper data integration


In [42]:
dim_product_df = pd.concat([prod_all, only_prod_in_file], ignore_index=True)

dim_product_df.to_sql("DimProduct", con=conn, index=False, if_exists="append")
print(f"Loaded DimProduct ({len(dim_product_df)} rows)")

# DimDate can load directly from staged file
dim_date_df = get_sql_dataframe("SELECT * FROM stg_dates_file", **mysql_args)
# Ensure correct typing for DateKey (int) and Date (date-like string is fine for MySQL)
dim_date_df.to_sql("DimDate", con=conn, index=False, if_exists="append")
print(f"Loaded DimDate ({len(dim_date_df)} rows)")

conn.close()

Loaded DimProduct (5 rows)
Loaded DimDate (10 rows)


/var/folders/4d/13vvrd7531qclhwg2svgst3c0000gn/T/ipykernel_33880/248840378.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dim_product_df = pd.concat([prod_all, only_prod_in_file], ignore_index=True)


### Build and Load Fact Table 

In [44]:
conn = get_sql_conn(**mysql_args)

stg_sales = get_sql_dataframe("SELECT * FROM stg_sales_file", **mysql_args)

cust_map = get_sql_dataframe("SELECT CustomerKey, CustomerID FROM DimCustomer", **mysql_args)
prod_map = get_sql_dataframe("SELECT ProductKey, ProductID FROM DimProduct", **mysql_args)
date_map = get_sql_dataframe("SELECT DateKey FROM DimDate", **mysql_args)

fact = stg_sales.merge(cust_map, on="CustomerID", how="left") \
                .merge(prod_map, on="ProductID", how="left")

missing_dates = set(fact["DateKey"]) - set(date_map["DateKey"])
if missing_dates:
    raise ValueError(f"DateKey values not in DimDate: {sorted(list(missing_dates))[:5]} ...")

fact_sales = fact[[
    "DateKey","CustomerKey","ProductKey","OrderID","Quantity","UnitPrice","TotalAmount"
]].copy()

fact_sales.to_sql("FactSales", con=conn, index=False, if_exists="append")

cnt = get_sql_dataframe("SELECT COUNT(*) AS cnt FROM FactSales", **mysql_args).iloc[0,0]
conn.close()

### Gross Margin Pipeline: join Mongo product_costs with FactSales and write FactSales_Margin

In [ ]:
# MongoDb 
product_costs_path = os.path.join(DATA_DIR, "product_costs.json")
product_costs = [
    {"ProductID": "SKU-100", "UnitCost": 12.00},
    {"ProductID": "SKU-101", "UnitCost": 55.00},
    {"ProductID": "SKU-102", "UnitCost": 14.00},
    {"ProductID": "SKU-200", "UnitCost":  7.25},
    {"ProductID": "SKU-300", "UnitCost":  4.50}
]
with open(product_costs_path, "w") as f:
    json.dump(product_costs, f, indent=2)

client = get_mongo_client(**mongodb_args)
json_files = {"product_costs": "product_costs.json"}
set_mongo_collections(client, mongodb_args["db_name"], DATA_DIR, json_files)

In [46]:
client = get_mongo_client(**mongodb_args)
costs_df = get_mongo_dataframe(client, mongodb_args["db_name"], "product_costs", {})

conn = get_sql_conn(**mysql_args)
dim_product = get_sql_dataframe("SELECT ProductKey, ProductID FROM DimProduct", **mysql_args)
fact_sales  = get_sql_dataframe("SELECT SalesKey, ProductKey, Quantity, TotalAmount FROM FactSales", **mysql_args)

# 2) Transform: join costs → compute margin
enriched = fact_sales.merge(dim_product, on="ProductKey", how="left") \
                     .merge(costs_df, on="ProductID", how="left")

# Defensive fill for missing costs (if any new SKUs appear later)
enriched["UnitCost"] = enriched["UnitCost"].fillna(0.0)

enriched["GrossMarginAmount"] = enriched["TotalAmount"] - (enriched["UnitCost"] * enriched["Quantity"])

# 3) Load: write a column-reduced analytics table (modifies columns vs source)
#    Only keep what analysts need for specific KPI—SalesKey + metric (requirement 2c)
out = enriched[["SalesKey", "GrossMarginAmount"]].copy()

set_dataframe(out, "FactSales_Margin", pk_column="SalesKey", db_operation="insert", **mysql_args)
conn.close()

## 3. Analytical Queries (Requirement 3)

**Demonstrates**: SELECT from 3+ tables with aggregation and GROUP BY operations. First, extract from file system and sql source system.


In [52]:
# 3.1 — Daily Revenue & Margin (Fact + DimDate + DimProduct)
q_daily_rev_margin = """
SELECT
  d.Date,
  ROUND(SUM(f.TotalAmount), 2)      AS Revenue,
  ROUND(SUM(m.GrossMarginAmount),2) AS GrossMargin
FROM FactSales f
JOIN DimDate d           ON f.DateKey = d.DateKey
JOIN DimProduct p        ON f.ProductKey = p.ProductKey
JOIN FactSales_Margin m  ON f.SalesKey = m.SalesKey
GROUP BY d.Date
ORDER BY d.Date;
"""
df_daily_rev_margin = get_sql_dataframe(q_daily_rev_margin, **mysql_args)
df_daily_rev_margin


,Date,Revenue,GrossMargin
0,2025-09-01,238.96,80.96
1,2025-09-02,76.50,33.00
2,2025-09-03,48.00,20.00
3,2025-09-05,79.92,43.92
4,2025-09-07,159.00,49.00
5,2025-09-08,96.00,40.00
6,2025-09-09,127.50,55.00
7,2025-09-10,79.94,37.94


In [53]:
# 3.2 — Revenue, Units & Margin by Date + Category (Fact + DimDate + DimProduct)
q_cat_by_day = """
SELECT
  d.Date,
  p.Category,
  SUM(f.Quantity)                    AS Units,
  ROUND(SUM(f.TotalAmount), 2)       AS Revenue,
  ROUND(SUM(m.GrossMarginAmount), 2) AS GrossMargin
FROM FactSales f
JOIN DimDate d           ON f.DateKey = d.DateKey
JOIN DimProduct p        ON f.ProductKey = p.ProductKey
JOIN FactSales_Margin m  ON f.SalesKey = m.SalesKey
GROUP BY d.Date, p.Category
ORDER BY d.Date, p.Category;
"""
df_cat_by_day = get_sql_dataframe(q_cat_by_day, **mysql_args)
df_cat_by_day


,Date,Category,Units,Revenue,GrossMargin
0,2025-09-01,Electronics,6.0,238.96,80.96
1,2025-09-02,Home & Kitchen,6.0,76.50,33.00
2,2025-09-03,Electronics,2.0,48.00,20.00
3,2025-09-05,Office,8.0,79.92,43.92
4,2025-09-07,Electronics,2.0,159.00,49.00
5,2025-09-08,Electronics,4.0,96.00,40.00
6,2025-09-09,Home & Kitchen,10.0,127.50,55.00
7,2025-09-10,Electronics,2.0,39.98,15.98
8,2025-09-10,Office,4.0,39.96,21.96


In [54]:
# 3.3 — Top Customers by Margin (Fact + DimCustomer + DimDate)
q_top_cust_margin = """
SELECT
  c.CustomerID,
  CONCAT(c.FirstName, ' ', c.LastName) AS CustomerName,
  ROUND(SUM(f.TotalAmount), 2)         AS Revenue,
  ROUND(SUM(m.GrossMarginAmount), 2)   AS GrossMargin,
  COUNT(DISTINCT f.OrderID)            AS OrderCount
FROM FactSales f
JOIN DimCustomer c      ON f.CustomerKey = c.CustomerKey
JOIN DimDate d          ON f.DateKey = d.DateKey
JOIN FactSales_Margin m ON f.SalesKey = m.SalesKey
GROUP BY c.CustomerID, CustomerName
ORDER BY GrossMargin DESC
LIMIT 10;
"""
df_top_cust_margin = get_sql_dataframe(q_top_cust_margin, **mysql_args)
df_top_cust_margin


,CustomerID,CustomerName,Revenue,GrossMargin,OrderCount
0,CUST-002,Liam Patel,286.50,104.00,2
1,CUST-001,Ava Nguyen,238.96,80.96,2
2,CUST-003,Sophia Kim,172.50,73.00,2
3,CUST-005,Mia Johnson,119.90,59.90,2
4,CUST-004,Noah Garcia,87.96,41.96,2


In [55]:
# 3.4 — AOV & AOM by Weekday (Fact + DimDate + DimCustomer)
q_aov_aom_weekday = """
-- AOV = Total Revenue / Distinct Orders
-- AOM = Total Margin  / Distinct Orders
SELECT
  d.DayOfWeek,
  ROUND(SUM(f.TotalAmount) / COUNT(DISTINCT f.OrderID), 2)       AS AvgOrderValue,
  ROUND(SUM(m.GrossMarginAmount) / COUNT(DISTINCT f.OrderID), 2) AS AvgOrderMargin
FROM FactSales f
JOIN DimDate d          ON f.DateKey = d.DateKey
JOIN DimCustomer c      ON f.CustomerKey = c.CustomerKey
JOIN FactSales_Margin m ON f.SalesKey = m.SalesKey
GROUP BY d.DayOfWeek
ORDER BY FIELD(d.DayOfWeek,'Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday');
"""
df_aov_aom_weekday = get_sql_dataframe(q_aov_aom_weekday, **mysql_args)
df_aov_aom_weekday


,DayOfWeek,AvgOrderValue,AvgOrderMargin
0,Monday,111.65,40.32
1,Tuesday,102.00,44.00
2,Wednesday,42.65,19.31
3,Friday,79.92,43.92
4,Sunday,159.00,49.00


In [56]:
# 3.5 — Product Leaderboard (Units, Revenue, Margin) (Fact + DimProduct + DimDate)
q_product_leaderboard = """
SELECT
  p.ProductID,
  p.ProductName,
  SUM(f.Quantity)                    AS UnitsSold,
  ROUND(SUM(f.TotalAmount), 2)       AS Revenue,
  ROUND(SUM(m.GrossMarginAmount), 2) AS GrossMargin
FROM FactSales f
JOIN DimProduct p        ON f.ProductKey = p.ProductKey
JOIN DimDate d           ON f.DateKey = d.DateKey
JOIN FactSales_Margin m  ON f.SalesKey = m.SalesKey
GROUP BY p.ProductID, p.ProductName
ORDER BY GrossMargin DESC, Revenue DESC, UnitsSold DESC;
"""
df_product_leaderboard = get_sql_dataframe(q_product_leaderboard, **mysql_args)
df_product_leaderboard


,ProductID,ProductName,UnitsSold,Revenue,GrossMargin
0,SKU-101,Mechanical Keyboard,4.0,318.00,98.00
1,SKU-200,Water Bottle 1L,16.0,204.00,88.00
2,SKU-300,Notebook Set (3),12.0,119.88,65.88
3,SKU-102,USB-C Charger,6.0,144.00,60.00
4,SKU-100,Wireless Mouse,6.0,119.94,47.94


In [57]:
# 3.6 — Monthly Rollup (Year/Month) Revenue & Margin (Fact + DimDate + DimProduct)
q_monthly_rollup = """
SELECT
  d.Year,
  d.Month,
  ROUND(SUM(f.TotalAmount), 2)       AS Revenue,
  ROUND(SUM(m.GrossMarginAmount), 2) AS GrossMargin
FROM FactSales f
JOIN DimDate d           ON f.DateKey = d.DateKey
JOIN DimProduct p        ON f.ProductKey = p.ProductKey
JOIN FactSales_Margin m  ON f.SalesKey = m.SalesKey
GROUP BY d.Year, d.Month
ORDER BY d.Year, d.Month;
"""
df_monthly_rollup = get_sql_dataframe(q_monthly_rollup, **mysql_args)
df_monthly_rollup


,Year,Month,Revenue,GrossMargin
0,2025,9,905.82,359.82
